# This is a Notebook to run SCNA analysis in the server

### Input is fastq files for this anaysis is low-coverage WGS data


What are you running:

- QC check using fastqc
- Alignment (mapping): bwa-aln + bwa samse
- Indexing
- multiqc check, optional
- QDNAseq (new version using hg38)

I am using it to have the steps for the SCNA analaysis in one notbeook as pipeline, and it can make it run form here but remember that it is running in th terminal


In [2]:
#in case you forgot where are you
!pwd
#!ls

/DATA/home/l.gonzalez/Notebooks


In [3]:
import os, sys
import glob
import logging

### Before starting the analysis we will check the QC of the fastq files usign fastqc
To do that, please go to the cell below and change the path to the folder where the fastq files are, then run it

In [ ]:
!fastqc /DATA/share/pipelines/lgleon/test/testCode_steps_Nousefuldata/testdata/fastq/*fastq.gz

##### As an optional step we can run multiqc after fastqc to create a unified html files with all the fastqc to see then in one go

## Run bwa for the mapping single end fastq files
There is no need for trimming


#Script:
#BWA ALN

#Ref
#/DATA/share/pipelines/lgleon/References/Ensembl/NGSfacility/Homo_sapiens.GRCh38.dna.primary_assembly.fa

#data
prj=/DATA/share/pipelines/lgleon/organoids_rawdata/DNA/B16PON/fastq


for file in ${prj}/*.f*q.gz
do

   inputfqgz=$file
   #outputbam=${file/%.fastq.gz/.bam}  #  % means that suffixes are matched, # would mean prefixes"
   filenamestem=${file/%.f*q.gz/}
   
   echo "!!!!!!!!!!!!!! Expected final output {$filenamestem.markdup.bam}"

   bwa aln -t 32 -n 2 \
    /DATA/share/pipelines/lgleon/References/Ensembl/NGSfacility/Homo_sapiens.GRCh38.dna.primary_assembly.fa  \
    $inputfqgz | bwa samse -r '@RG\tID:inputfqgz\tSM:inputfqgz' \
    /DATA/share/pipelines/lgleon/References/Ensembl/NGSfacility/Homo_sapiens.GRCh38.dna.primary_assembly.fa \
    - $inputfqgz | samtools view -Su - | samtools sort - -o $filenamestem.sorted.bam > $filenamestem.sorted.bam

   picard MarkDuplicates -I $filenamestem.sorted.bam -O $filenamestem.markdup.bam -M $filenamestem.markdup_metrics.txt -ASSUME_SORTED true -CREATE_INDEX true -VALIDATION_STRINGENCY LENIENT

done

                        

mkdir fastQC
#mv *f*q.gz fastQC

mkdir sorted_bam
mv *sorted.bam sorted_bam
#rm *sorted.bam

mkdir markdup_bam
mv *markdup.bam markdup_bam  
    

In [14]:
## Alignment with bwa aln, code is at /home/l.gonzalez/Notebooks/bash_R_scripts/alternatve_bwaAln_hg38_lgl.sh
## Go to the folder where you want to save the results
## Check in the code, reference is hard copied
## path to the fastq files must be change for each study or anaylsis
## this code I meant to be run from the same folder where teh fastq files are store
### Remember the project folder IS HARDCODE, need to be changed
# once you chack the folder, just the cell 

!bash /DATA/home/l.gonzalez/Notebooks/bash_R_scripts/alternatve_bwaAln_hg38_lgl.sh


!!!!!!!!!!!!!! Expected final output {/DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/fastq/5564_10_Tissue_pt_264_2_S40_R1_001.markdup.bam}
[bwa_aln_core] calculate SA coordinate... 44.87 sec
[bwa_aln_core] write to the disk... 0.05 sec
[bwa_aln_core] 262144 sequences have been processed.
[bwa_aln_core] calculate SA coordinate... 45.05 sec
[bwa_aln_core] write to the disk... [bwa_aln_core] convert to sequence coordinate... 4.03 sec
[bwa_aln_core] refine gapped alignments... 0.63 sec
[bwa_aln_core] print alignments... 0.31 sec
[bwa_aln_core] 262144 sequences have been processed.
0.06 sec
[bwa_aln_core] 524288 sequences have been processed.
[bwa_aln_core] calculate SA coordinate... 44.57 sec
[bwa_aln_core] write to the disk... [bwa_aln_core] convert to sequence coordinate... 4.04 sec
[bwa_aln_core] refine gapped alignments... 0.63 sec
[bwa_aln_core] print alignments... 0.31 sec
[bwa_aln_core] 524288 sequences have been processed.
0.05 sec
[bwa_aln_core] 786432 sequences have been proc

INFO	2021-05-07 17:05:36	MarkDuplicates	Read     7,000,000 records.  Elapsed time: 00:00:48s.  Time for last 1,000,000:    5s.  Last read position: X:110,025,519
INFO	2021-05-07 17:05:36	MarkDuplicates	Tracking 0 as yet unmatched pairs. 0 records in RAM.
Ignoring SAM validation error: ERROR: Record 7098523, Read name D00645:375:CDYJNANXX:4:1108:7809:17181, MAPQ should be 0 for unmapped read.
Ignoring SAM validation error: ERROR: Record 7145316, Read name D00645:375:CDYJNANXX:4:2108:6695:73950, MAPQ should be 0 for unmapped read.
INFO	2021-05-07 17:05:37	MarkDuplicates	Read 7148943 records. 0 pairs never matched.
INFO	2021-05-07 17:05:37	MarkDuplicates	After buildSortedReadEndLists freeMemory: 731951176; totalMemory: 757596160; maxMemory: 954728448
INFO	2021-05-07 17:05:37	MarkDuplicates	Will retain up to 29835264 duplicate indices before spilling to disk.
INFO	2021-05-07 17:05:37	MarkDuplicates	Traversing read pair information and detecting duplicates.
INFO	2021-05-07 17:05:37	MarkDupl

INFO	2021-05-07 17:10:28	MarkDuplicates	Before output close freeMemory: 826479568; totalMemory: 839909376; maxMemory: 954728448
INFO	2021-05-07 17:10:28	MarkDuplicates	After output close freeMemory: 827003856; totalMemory: 840433664; maxMemory: 954728448
[Fri May 07 17:10:28 CEST 2021] picard.sam.markduplicates.MarkDuplicates done. Elapsed time: 1.11 minutes.
Runtime.totalMemory()=840433664
!!!!!!!!!!!!!! Expected final output {/DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/fastq/5564_5_Tissue_pt_120_2_S35_R1_001.markdup.bam}
[bwa_aln_core] calculate SA coordinate... 40.58 sec
[bwa_aln_core] write to the disk... 0.05 sec
[bwa_aln_core] 262144 sequences have been processed.
[bwa_aln_core] calculate SA coordinate... 39.71 sec
[bwa_aln_core] write to the disk... [bwa_aln_core] convert to sequence coordinate... 4.04 sec
[bwa_aln_core] refine gapped alignments... 0.64 sec
[bwa_aln_core] print alignments... 0.31 sec
[bwa_aln_core] 262144 sequences have been processed.
0.05 sec
[bwa_aln_c

[main] Version: 0.7.17-r1188
[main] CMD: bwa aln -t 32 -n 2 /DATA/share/pipelines/lgleon/References/Ensembl/NGSfacility/Homo_sapiens.GRCh38.dna.primary_assembly.fa /DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/fastq/5564_5_Tissue_pt_120_2_S35_R1_001.fastq.gz[bwa_aln_core] convert to sequence coordinate... 
[main] Real time: 162.845 sec; CPU: 877.089 sec
3.20 sec
[bwa_aln_core] refine gapped alignments... 0.53 sec
[bwa_aln_core] print alignments... 0.11 sec
[bwa_aln_core] 5601490 sequences have been processed.
[main] Version: 0.7.17-r1188
[main] CMD: bwa samse -r @RG\tID:inputfqgz\tSM:inputfqgz /DATA/share/pipelines/lgleon/References/Ensembl/NGSfacility/Homo_sapiens.GRCh38.dna.primary_assembly.fa - /DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/fastq/5564_5_Tissue_pt_120_2_S35_R1_001.fastq.gz
[main] Real time: 166.710 sec; CPU: 117.435 sec
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
INFO	2021-05-07 17:13:34	MarkDuplicates	

********** NOTE: Picard's command l

[bwa_aln_core] convert to sequence coordinate... 3.86 sec
[bwa_aln_core] refine gapped alignments... 0.60 sec
[bwa_aln_core] print alignments... 0.29 sec
[bwa_aln_core] 6807610 sequences have been processed.
[main] Version: 0.7.17-r1188
[main] CMD: bwa samse -r @RG\tID:inputfqgz\tSM:inputfqgz /DATA/share/pipelines/lgleon/References/Ensembl/NGSfacility/Homo_sapiens.GRCh38.dna.primary_assembly.fa - /DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/fastq/5564_6_Tissue_pt_137_2_S36_R1_001.fastq.gz
[main] Real time: 206.995 sec; CPU: 133.809 sec
[bam_sort_core] merging from 2 files and 1 in-memory blocks...
INFO	2021-05-07 17:18:36	MarkDuplicates	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    MarkDuplicates -I 

...

0.06 sec
[bwa_aln_core] 6815744 sequences have been processed.
[bwa_aln_core] calculate SA coordinate... 69.08 sec
[bwa_aln_core] write to the disk... [bwa_aln_core] convert to sequence coordinate... 4.04 sec
[bwa_aln_core] refine gapped alignments... 0.60 sec
[bwa_aln_core] print alignments... 0.27 sec
[bwa_aln_core] 6815744 sequences have been processed.
0.06 sec
[bwa_aln_core] 7077888 sequences have been processed.
[bwa_aln_core] calculate SA coordinate... 68.30 sec
[bwa_aln_core] write to the disk... [bwa_aln_core] convert to sequence coordinate... 3.90 sec
[bwa_aln_core] refine gapped alignments... 0.60 sec
[bwa_aln_core] print alignments... 0.27 sec
[bwa_aln_core] 7077888 sequences have been processed.
0.05 sec
[bwa_aln_core] 7340032 sequences have been processed.
[bwa_aln_core] calculate SA coordinate... 67.21 sec
[bwa_aln_core] write to the disk... [bwa_aln_core] convert to sequence coordinate... 4.00 sec
[bwa_aln_core] refine gapped alignments... 0.60 sec
[bwa_aln_core] print 

INFO	2021-05-07 20:13:16	MarkDuplicates	Start of doWork freeMemory: 502528272; totalMemory: 514850816; maxMemory: 954728448
INFO	2021-05-07 20:13:16	MarkDuplicates	Reading input file and constructing read end information.
INFO	2021-05-07 20:13:16	MarkDuplicates	Will retain up to 3459161 data points before spilling to disk.
INFO	2021-05-07 20:13:24	MarkDuplicates	Read     1,000,000 records.  Elapsed time: 00:00:07s.  Time for last 1,000,000:    7s.  Last read position: 11:13,109,374
INFO	2021-05-07 20:13:24	MarkDuplicates	Tracking 0 as yet unmatched pairs. 0 records in RAM.
INFO	2021-05-07 20:13:29	MarkDuplicates	Read     2,000,000 records.  Elapsed time: 00:00:12s.  Time for last 1,000,000:    5s.  Last read position: 14:81,545,177
INFO	2021-05-07 20:13:29	MarkDuplicates	Tracking 0 as yet unmatched pairs. 0 records in RAM.
INFO	2021-05-07 20:13:34	MarkDuplicates	Read     3,000,000 records.  Elapsed time: 00:00:17s.  Time for last 1,000,000:    4s.  Last read position: 19:41,252,976
INF

[bwa_aln_core] calculate SA coordinate... 57.08 sec
[bwa_aln_core] write to the disk... [bwa_aln_core] convert to sequence coordinate... 4.01 sec
[bwa_aln_core] refine gapped alignments... 0.61 sec
[bwa_aln_core] print alignments... 0.29 sec
[bwa_aln_core] 2097152 sequences have been processed.
0.06 sec
[bwa_aln_core] 2359296 sequences have been processed.
[bwa_aln_core] calculate SA coordinate... 59.56 sec
[bwa_aln_core] write to the disk... [bwa_aln_core] convert to sequence coordinate... 4.08 sec
[bwa_aln_core] refine gapped alignments... 0.64 sec
[bwa_aln_core] print alignments... 0.31 sec
[bwa_aln_core] 2359296 sequences have been processed.
0.05 sec
[bwa_aln_core] 2621440 sequences have been processed.
[bwa_aln_core] calculate SA coordinate... 57.51 sec
[bwa_aln_core] write to the disk... [bwa_aln_core] convert to sequence coordinate... 3.97 sec
[bwa_aln_core] refine gapped alignments... 0.60 sec
[bwa_aln_core] print alignments... 0.29 sec
[bwa_aln_core] 2621440 sequences have be

0.05 sec
[bwa_aln_core] 8126464 sequences have been processed.
[bwa_aln_core] calculate SA coordinate... 59.73 sec
[bwa_aln_core] write to the disk... [bwa_aln_core] convert to sequence coordinate... 4.52 sec
[bwa_aln_core] refine gapped alignments... 0.67 sec
[bwa_aln_core] print alignments... 0.35 sec
[bwa_aln_core] 8126464 sequences have been processed.
0.05 sec
[bwa_aln_core] 8388608 sequences have been processed.
[bwa_aln_core] calculate SA coordinate... 58.49 sec
[bwa_aln_core] write to the disk... [bwa_aln_core] convert to sequence coordinate... 4.67 sec
[bwa_aln_core] refine gapped alignments... 0.79 sec
[bwa_aln_core] print alignments... 0.32 sec
[bwa_aln_core] 8388608 sequences have been processed.
0.07 sec
[bwa_aln_core] 8650752 sequences have been processed.
[bwa_aln_core] calculate SA coordinate... 52.18 sec
[bwa_aln_core] write to the disk... [bwa_aln_core] convert to sequence coordinate... 4.35 sec
[bwa_aln_core] refine gapped alignments... 0.64 sec
[bwa_aln_core] print 

INFO	2021-05-07 20:22:04	MarkDuplicates	Start of doWork freeMemory: 502528272; totalMemory: 514850816; maxMemory: 954728448
INFO	2021-05-07 20:22:04	MarkDuplicates	Reading input file and constructing read end information.
INFO	2021-05-07 20:22:04	MarkDuplicates	Will retain up to 3459161 data points before spilling to disk.
INFO	2021-05-07 20:22:10	MarkDuplicates	Read     1,000,000 records.  Elapsed time: 00:00:06s.  Time for last 1,000,000:    6s.  Last read position: 10:68,107,769
INFO	2021-05-07 20:22:10	MarkDuplicates	Tracking 0 as yet unmatched pairs. 0 records in RAM.
INFO	2021-05-07 20:22:15	MarkDuplicates	Read     2,000,000 records.  Elapsed time: 00:00:11s.  Time for last 1,000,000:    4s.  Last read position: 12:95,357,273
INFO	2021-05-07 20:22:15	MarkDuplicates	Tracking 0 as yet unmatched pairs. 0 records in RAM.
INFO	2021-05-07 20:22:20	MarkDuplicates	Read     3,000,000 records.  Elapsed time: 00:00:16s.  Time for last 1,000,000:    5s.  Last read position: 15:77,385,573
INF

#### Check the results, 
put them in the appropiate folders. Folder should be created in the process, but better check



### Indexing the bam files to create the bai, otherwise we can not continue

#script
#!/bin/bash
#prj=/DATA/share/pipelines/lgleon/organoids_rawdata/DNA/markdup
prj=/DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/fastq
for i in ${prj}/*markdup.bam

do

echo "indexing: "$i
samtools index $i $i".bai"

done

In [16]:
# for Indexing the bam I have a tiny script that work quite nicely
### Remember the project folder IS HARDCODE, need to be changed

##WORKS

!bash /DATA/home/l.gonzalez/Notebooks/bash_R_scripts/indexing.sh

indexing: /DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/markdup/5564_10_Tissue_pt_264_2_S40_R1_001.markdup.bam
indexing: /DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/markdup/5564_2_Tissue_pt_83_2_S32_R1_001.markdup.bam
indexing: /DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/markdup/5564_5_Tissue_pt_120_2_S35_R1_001.markdup.bam
indexing: /DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/markdup/5564_6_Tissue_pt_137_2_S36_R1_001.markdup.bam
indexing: /DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/markdup/5564_9_Tissue_pt_216_2_S39_R1_001.markdup.bam
indexing: /DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/markdup/5710_1_74_CTGATCGT_S2_R1_001.markdup.bam
indexing: /DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/markdup/5710_19_88_CTTAGGAC_S19_R1_001.markdup.bam
indexing: /DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/markdup/5710_5_152_CTTGTCGA_S9_R1_001.markdup.bam
indexing: /DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/markdup/5867_10_110_TAACCGGT_S5

### After the analysis we will check the QC of the steps that we ran
To do that, please go to the cell below and change the path to the folders where the fastqc files are, also the bam/bai files and multiqc will create a html with all the files

In [ ]:
# check and change the path to the folder then click run:
!multiqc /DATA/share/pipelines/lgleon/test/testCode_steps_Nousefuldata/testdata/fastq/

## Run QDNAseq for the calling, 
This new script, modified form Erik run from bam to called 
The CLI can be run directly but using R

The code have some hardcode parts, path to project directory(output) and path to bam files (input)
If you wanna run it with hg38, not need to change that part
For the bin size it should be change also inside the code, for the calling and devawing 

To make those cahnges, there are three options:
- Open the file here using the notebooks 
- Open in Rstudio  
- Open in the terminal

Since it is a Rmarkdown file there are also two ways to run it:
- Inside the Rstudio, step/chunk by step/chunk
- From terminal (easier)

In [18]:
# To run as CLI
# Here is the example of the NGSprotocol data as an example, DO NOT RUN IT
!R -e "rmarkdown::render('/DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/qdnaseq_output/QDNASeqFlow_cergentis-hg38_30Kb.Rmd')"
#!/opt/R/4.0.4/bin/Rscript -e "rmarkdown::render('/DATA/share/pipelines/lgleon/NGS_protocol/results_QDNAseq/normalS/QDNASeqFlow_hg38_30Kb.Rmd')"

#The General Code, which is not, it is at 
# !R -e "/home/l.gonzalez/Notebooks/bash_R_scripts/QDNASeqFlow_hg38_NGSprotocol_30Kb.Rmd"

#The best way to work is make a copy of the general one, in the results folder, and run it form there
#as a knit it will create a html file at the same time




R version 3.6.3 (2020-02-29) -- "Holding the Windsock"
Copyright (C) 2020 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> rmarkdown::render('/DATA/share/pipelines/lgleon/cergentis_C5/Raw_data/qdnaseq_output/QDNASeqFlow_cergentis-hg38_30Kb.Rmd')


processing file: QDNASeqFlow_cergentis-hg38_30Kb.Rmd
  |..                                                                    |   3%
  ordinary text without R code

  |.....    

451515658365034241.2445.37463545108359326398.6826.87463545135449157398.61033.4
451515458365031241.2445.37463545108359326398.6826.87463545135449157398.61033.4
451515758944934241.2449.87463545108359326398.6826.87463545135449157398.61033.4
451515258282221241.2444.77463545108359326398.6826.87463545135449157398.61033.4
451516158282225241.2444.77463545108359326398.6826.87463545135449157398.61033.4
451517058282229241.2444.77463545108359326398.6826.87463545135449157398.61033.4
451517958282233241.2444.77463545108359326398.6826.87463545135449157398.61033.4
451518858282237241.2444.77463545108359326398.6826.87463545135449157398.61033.4
451519458282240241.2444.77463545108359326398.6826.87463545135449157398.61033.4
451521558282278241.2444.77463545108359326398.6826.87463545135449157398.61033.4
451514860021865241.24587463545108359326398.6826.87463545135449157398.61033.4
4
451515463004050241.2480.77463545108359326398.6826.87463545135449157398.61033.4
451515663004053241.2480.77463545108359326398.6826.87

In [3]:
#! selectR-config --select 3.6
#! selectR-config --select 4.0

[1] TRUE


In [ ]:
#!R
#!/opt/R/4.0.4/bin/Rscript


R version 3.6.3 (2020-02-29) -- "Holding the Windsock"
Copyright (C) 2020 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> 

In [4]:
#!ls /usr/local/bin/


alignmentSieve		  google-oauthlib-tool	    pyrsa-keygen
bamCompare		  import_pb_to_tensorboard  pyrsa-priv2pub
bamCoverage		  markdown_py		    pyrsa-sign
bamPEFragmentSize	  multiBamSummary	    pyrsa-verify
bigwigCompare		  multiBigwigSummary	    saved_model_cli
computeGCBias		  pbr			    sphinx-apidoc
computeMatrix		  plotCorrelation	    sphinx-autogen
computeMatrixOperations   plotCoverage		    sphinx-build
correctGCBias		  plotEnrichment	    sphinx-quickstart
deeptools		  plotFingerprint	    tensorboard
estimateReadFiltering	  plotHeatmap		    tflite_convert
estimateScaleFactor	  plotPCA		    tf_upgrade_v2
estimator_ckpt_converter  plotProfile		    toco
f2py			  pybabel		    toco_from_protos
f2py3			  pyrsa-decrypt		    tqdm
f2py3.8			  pyrsa-encrypt		    wheel
